In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import math 
from tensorflow.keras.layers import Flatten, Dense, Dropout, Activation
from tensorflow.keras.models import Sequential
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras import optimizers
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import KNNImputer
from numpy import linalg as LA
import math
import re
import math
import os
from tempfile import TemporaryDirectory
from typing import Tuple
import copy
import time
import torch.optim as optim
import torch
from torch import nn, Tensor
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torch.nn import MultiheadAttention
from torch.utils.data import dataset

ModuleNotFoundError: No module named 'torch'

In [59]:
N_rows = 1000000
labe=pd.read_csv('labevents.csv',nrows=N_rows)
labeinteret=labe.copy()
labeinteret=labeinteret[['subject_id','specimen_id','itemid','charttime','storetime','valuenum','ref_range_lower','ref_range_upper','flag','priority']]
labeinteret['intcharttime']=[int(re.sub(r"\D", "", x)) if isinstance(x,str) else x for x in labeinteret['charttime']]
labeinteret['intstoretime']=[int(re.sub(r"\D", "", x)) if isinstance(x,str) else x for x in labeinteret['storetime']]
labeinteret=labeinteret.drop(columns=['charttime','storetime'],axis=1)
labeinteret=labeinteret.drop(columns=['flag','priority'],axis=1)
labeinteret=labeinteret.dropna()


In [60]:
labeinteret.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 783208 entries, 0 to 999994
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   subject_id       783208 non-null  int64  
 1   specimen_id      783208 non-null  int64  
 2   itemid           783208 non-null  int64  
 3   valuenum         783208 non-null  float64
 4   ref_range_lower  783208 non-null  float64
 5   ref_range_upper  783208 non-null  float64
 6   intcharttime     783208 non-null  int64  
 7   intstoretime     783208 non-null  float64
dtypes: float64(4), int64(4)
memory usage: 53.8 MB


In [61]:
Nrest=783208
labeinteretdatestr= labeinteret['intcharttime'].astype(str)
labeinteret['yrmeasure']=np.zeros(Nrest)
labeinteret['monthmeasure']=np.zeros(Nrest)
labeinteret['daymeasure']=np.zeros(Nrest)
labeinteret['hrmeasure']=np.zeros(Nrest)
labeinteret['minmeasure']=np.zeros(Nrest)
labeinteret['minmeasure']=np.zeros(Nrest)
labeinteret['secmeasure']=np.zeros(Nrest)

for k in range(783208):
    if k in labeinteretdatestr :
        labeinteret['yrmeasure'][k]=int(labeinteretdatestr[k][:4])
        labeinteret['monthmeasure'][k]=int(labeinteretdatestr[k][4:6])
        labeinteret['daymeasure'][k]=int(labeinteretdatestr[k][6:8])
        labeinteret['hrmeasure'][k]=int(labeinteretdatestr[k][8:10])
        labeinteret['minmeasure'][k]=int(labeinteretdatestr[k][10:12])
        labeinteret['secmeasure'][k]=int(labeinteretdatestr[k][12:14])
        
labeinteret['yrmeasure']=labeinteret['yrmeasure'].astype(int)
labeinteret= labeinteret.sort_values(['subject_id','yrmeasure','monthmeasure','daymeasure','hrmeasure','minmeasure'])
labeinteret = labeinteret.drop('intstoretime',axis=1)
labeinteret = labeinteret.drop('secmeasure',axis=1)
labeinteret = labeinteret.drop('minmeasure',axis=1)
labeinteret = labeinteret.drop('hrmeasure',axis=1)
labeinteret = labeinteret.drop('daymeasure',axis=1)
#labeinteret = labeinteret.drop(['ref_range_lower','ref_range_upper','yrmeasure','monthmeasure'],axis=1)
labeinteretarray= labeinteret.to_numpy()


/var/folders/j0/n1zmwvf923q5h9v_4n2k33100000gn/T/ipykernel_10407/3026321972.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labeinteret['yrmeasure'][k]=int(labeinteretdatestr[k][:4])
/var/folders/j0/n1zmwvf923q5h9v_4n2k33100000gn/T/ipykernel_10407/3026321972.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labeinteret['monthmeasure'][k]=int(labeinteretdatestr[k][4:6])
/var/folders/j0/n1zmwvf923q5h9v_4n2k33100000gn/T/ipykernel_10407/3026321972.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-

In [62]:
listindiv = list(set(labeinteretarray[:,0]))

In [66]:
sepers =[]
for i in range(len(listindiv)) :
    sepers.append([])
for x in labeinteretarray :
    sepers[listindiv.index(x[0])].append(x)
    

In [68]:
[len(x) for x in sepers].index(max([len(x) for x in sepers]))

1212

In [69]:
meclong=np.array(sepers[1212])


In [70]:
listspec=list(set(meclong[:,2]))

In [71]:
sepersanal =[]
for i in range(len(listspec)) :
    sepersanal.append([])
for x in meclong :
    sepersanal[listspec.index(x[2])].append(x)
    

In [82]:
[len(x) for x in sepersanal].index(max([len(x) for x in sepersanal]))

18

In [37]:
labeinteret.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 78528 entries, 0 to 99999
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   subject_id       78528 non-null  int64  
 1   specimen_id      78528 non-null  int64  
 2   itemid           78528 non-null  int64  
 3   valuenum         78528 non-null  float64
 4   ref_range_lower  78528 non-null  float64
 5   ref_range_upper  78528 non-null  float64
 6   intcharttime     78528 non-null  int64  
 7   yrmeasure        78528 non-null  int64  
 8   monthmeasure     78528 non-null  float64
dtypes: float64(4), int64(5)
memory usage: 6.0 MB


In [74]:
len(sepers[1212])

14072

In [76]:
h=list(set(labeinteretarray[:,0]))
tableau=[]
for k in range(len(h)) :
    tableau.append([])
for x in labeinteretarray :
    tableau[h.index(x[0])].append(x)

In [77]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=0.1)
        
        # Compute the positional encodings once in log space
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        # Add the positional encodings in the time dimension
        x = x + self.pe[:, :x.size(1)]
        return self.dropout(x)

In [88]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

# Define the transformer model
class TransformerModel(nn.Module):
    def __init__(self, input_dim, output_dim, nhead, num_layers):
        super(TransformerModel, self).__init__()
        self.pos = PositionalEncoding(d_model=4)
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(input_dim, nhead), num_layers
        )
        self.linear = nn.Linear(input_dim * 24, output_dim)

    def forward(self, x):
        x = self.pos(x)
        # x has shape (seq_len, batch, input_dim)
        x = self.transformer(x)
        # x has shape (seq_len, batch, input_dim)
        x = x.transpose(0, 1)
        # x has shape (batch, seq_len, input_dim)
        x = x.flatten(start_dim=1)
        # x has shape (batch, seq_len * input_dim)
        x = self.linear(x)
        # x has shape (batch, output_dim)
        return x

# Define the training loop
def train(model, optimizer, criterion, X, y, batch_size, num_epochs):
    for epoch in range(num_epochs):
        total_loss = 0
        for i in range(0, len(X) - batch_size, batch_size):
            optimizer.zero_grad()
            inputs = torch.tensor(X[i:i+batch_size]).float().transpose(0, 1)
            targets = torch.tensor(y[i:i+batch_size]).float()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            total_loss += loss.item() * batch_size
        print("Epoch {}, Loss: {}".format(epoch+1, total_loss / len(X)))



In [85]:

# Generate some random data
data = np.array(sepersanal[18])
data = data[:,5:]
data_mean = np.mean(data,axis=0)
data_std = np.std(data,axis=0)
data = (data - data_mean) / data_std

In [86]:
data

array([[ 0.98311097, -1.419208  , -1.39724612,  0.38909599],
       [ 0.98311097, -1.41908893, -1.39724612,  0.38909599],
       [ 0.98311097, -1.41908093, -1.39724612,  0.38909599],
       ...,
       [-1.01717917,  1.31648074,  1.28624993, -0.17202139],
       [-1.01717917,  1.31648658,  1.28624993, -0.17202139],
       [-1.01717917,  1.31660739,  1.28624993, -0.17202139]])

In [89]:

# Split the data into input and output sequences
X, y = [], []
for i in range(24, len(data)):
    X.append(data[i-24:i])
    y.append(data[i])
X = np.array(X)
y = np.array(y)
    # Split the data into training and validation sets
split = int(0.8 * len(X))
X_train, y_train = X[:split], y[:split]
X_val, y_val = X[split:], y[split:]
    # Define the model, optimizer, and loss function
model = TransformerModel(4, 4, nhead=2, num_layers=2)
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.MSELoss()
    # Train the model
train(model, optimizer, criterion, X_train, y_train, batch_size=16, num_epochs=50)
    # Evaluate the model on the validation set
inputs = torch.tensor(X_val).float().transpose(0, 1)
targets = torch.tensor(y_val).float()
outputs = model(inputs)
loss = criterion(outputs, targets)
print("Validation Loss:", loss.item())


Epoch 1, Loss: 1.1650974904835032
Epoch 2, Loss: 2.1230349957364276
Epoch 3, Loss: 1.55009297954226
Epoch 4, Loss: 0.9900228722581585
Epoch 5, Loss: 1.0037159765422536
Epoch 6, Loss: 0.9458726463194418
Epoch 7, Loss: 0.8372472968302113
Epoch 8, Loss: 0.6013258930934672
Epoch 9, Loss: 0.763569129709287
Epoch 10, Loss: 1.1263110822844273
Epoch 11, Loss: 0.9359262413963145
Epoch 12, Loss: 0.700408707158851
Epoch 13, Loss: 0.6784135176525918
Epoch 14, Loss: 0.6016466208646213
Epoch 15, Loss: 0.5857152784526541
Epoch 16, Loss: 0.5329817730246238
Epoch 17, Loss: 0.5318879340458842
Epoch 18, Loss: 0.47384554359905157
Epoch 19, Loss: 0.3578713534333559
Epoch 20, Loss: 0.489583560178195
Epoch 21, Loss: 0.634960993979741
Epoch 22, Loss: 0.5117061300185121
Epoch 23, Loss: 0.5988904974607202
Epoch 24, Loss: 0.3800906630588581
Epoch 25, Loss: 0.24753141866146938
Epoch 26, Loss: 0.2783279970625843
Epoch 27, Loss: 0.22516813710283692
Epoch 28, Loss: 0.27180234822640525
Epoch 29, Loss: 0.1952942802682

In [90]:
sortie=outputs

sortiexpct=y_val
for k in range(len(sortie)) :
    print(sortie[k])
    print(sortiexpct[k])

tensor([-0.6299,  0.9090,  0.9480, -0.6599], grad_fn=<SelectBackward0>)
[-1.01717917  1.286105    1.28624993 -0.73313877]
tensor([-0.1364,  0.5756,  0.5728, -0.6740], grad_fn=<SelectBackward0>)
[-1.01717917  1.28623583  1.28624993 -0.73313877]
tensor([ 0.0449,  0.4852,  0.5663, -0.5580], grad_fn=<SelectBackward0>)
[-1.01717917  1.28637391  1.28624993 -0.73313877]
tensor([-0.8305,  1.0850,  1.0687, -0.8772], grad_fn=<SelectBackward0>)
[-1.01717917  1.28651286  1.28624993 -0.73313877]
tensor([-0.6149,  0.7549,  0.7575, -0.3637], grad_fn=<SelectBackward0>)
[-1.01717917  1.28652805  1.28624993 -0.73313877]
tensor([-0.7989,  0.8809,  0.8572, -0.4208], grad_fn=<SelectBackward0>)
[-1.01717917  1.28665105  1.28624993 -0.73313877]
tensor([-0.8858,  0.9358,  0.9144, -0.5299], grad_fn=<SelectBackward0>)
[-1.01717917  1.28707665  1.28624993 -0.73313877]
tensor([-0.2462,  0.7912,  0.8170, -0.7766], grad_fn=<SelectBackward0>)
[-1.01717917  1.28762691  1.28624993 -0.73313877]
tensor([-0.7216,  0.9444